# **Образец получения статистики**

In [1]:
# перед началом работы импортируем библиотеки, которые могут понадобиться в работе над проектом

import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats as st
import seaborn as sns
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import requests 
from urllib.parse import urlencode 
import re
from calendar import day_name

%matplotlib inline
# импорт библиотеки подключения к яндекс-метрике
from tapi_yandex_metrika import YandexMetrikaStats
from pandas.io.json import json_normalize
import json
from requests_html import HTMLSession
session = HTMLSession()
from pandas import Series,DataFrame
from io import BytesIO
from time import sleep
import sys

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [5]:
# Метод для корректной обработки строк в кодировке UTF-8 как в Python 3, так и в Python 2
if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode("utf8")
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

## Ссылка на спецификации запросов директа

https://yandex.ru/dev/direct/doc/reports/spec.html

https://yandex.ru/dev/direct/doc/reports/fields-list.html

In [6]:
# токен доступа к директу
ACCESS_TOKEN = "!!!!YOUR_TOKEN!!!!!"


In [20]:
# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

#адрес сервиса ОТЧЕТОВ ПЕСОЧНИЦА
ReportsURL_SANDBOX = 'https://api-sandbox.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = ACCESS_TOKEN

# Логин клиента рекламного агентства
# Обязательный параметр, если запросы выполняются от имени рекламного агентства
#clientLogin = 'ЛОГИН_КЛИЕНТА'

#какие столбцы подгрузить
stolbec = ["Date", "CampaignName", "AdGroupName",  "LocationOfPresenceName", "AdNetworkType", "Device",
           "Impressions", "Clicks", "Cost",  "Bounces", "AvgPageviews", "Conversions"]

#ID цели
#goalID = ['43693399'] # если целей несколько то ['123','324']

#даты отчета
StartDate = "2022-01-01"
FinishDate = "2022-01-02"

# --- Подготовка запроса ---
# Создание HTTP-заголовков запроса
headers = {
           # OAuth-токен. Использование слова Bearer обязательно
           "Authorization": "Bearer " + token,
           # Логин клиента рекламного агентства
          # "Client-Login": clientLogin,
           # Язык ответных сообщений
           "Accept-Language": "ru",
           # Режим формирования отчета
           "processingMode": "auto",
           # Формат денежных значений в отчете, если не сделать, то расходы будут в миллионах
            "returnMoneyInMicros": "false",
           # Не выводить в отчете строку с названием отчета и диапазоном дат
           # "skipReportHeader": "true",
           # Не выводить в отчете строку с названиями полей
           # "skipColumnHeader": "true",
           # Не выводить в отчете строку с количеством строк статистики
            "skipReportSummary": "true"
           }

# Создание тела запроса
body = {
    "params": {
        # критерии выбора, тут могут быть фильтры, логические операторы, 
        "SelectionCriteria": {
            "DateFrom": StartDate,
            "DateTo": FinishDate
                             },
        # цель, если требуется
        #"Goals": goalID, 
        
        "FieldNames": stolbec,
        "ReportName": u("Отчет 99"),
        "ReportType": "CUSTOM_REPORT",   # тип отчета
        "DateRangeType": "CUSTOM_DATE",
        "Format": "TSV",  # обязательно
        "IncludeVAT": "YES",
        "IncludeDiscount": "NO"
              }
       }

# Кодирование тела запроса в JSON
body = json.dumps(body, indent=4)


## Функция запроса к директу (req_to_direct)

In [17]:
def req_to_direct(URL, body, headers):   
    # --- Запуск цикла для выполнения запросов ---
    # Если получен HTTP-код 200, то выводится содержание отчета
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы
    while True:
        try:
            req = requests.post(URL, body, headers=headers)
            req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print("Параметры запроса указаны неверно или достигнут лимит отчетов в очереди")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(u(body)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 200:
                print("Отчет создан успешно")
                format(u(req.text))
                return(req.text)
                # return(req.content) если вдруг хотим в байтах выгрузить и преобразовывать
                break
            elif req.status_code == 201:
                print("Отчет успешно поставлен в очередь в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print("Отчет формируется в режиме офлайн")
                retryIn = int(req.headers.get("retryIn", 60))
                print("Повторная отправка запроса через {} секунд".format(retryIn))
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее")
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            elif req.status_code == 502:
                print("Время формирования отчета превысило серверное ограничение.")
                print("Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                print("JSON-код запроса: {}".format(body))
                print("RequestId: {}".format(req.headers.get("RequestId", False)))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break
            else:
                print("Произошла непредвиденная ошибка")
                print("RequestId:  {}".format(req.headers.get("RequestId", False)))
                print("JSON-код запроса: {}".format(body))
                print("JSON-код ответа сервера: \n{}".format(u(req.json())))
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print("Произошла ошибка соединения с сервером API")
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print("Произошла непредвиденная ошибка")
            # Принудительный выход из цикла
            break

In [21]:
result = req_to_direct(ReportsURL_SANDBOX, body, headers)

# Преобразование ответа в датафрейм   ()      
file = open("cashe.csv", "w")   # сздаем и открываем новый файл
file.write(result)            # записываем в него ответ API
file.close()                    # закрываем файл
df = pd.read_csv("cashe.csv", header=1, sep='\t', index_col=0, encoding='cp1251')     # записываем в датафрейм файл
df = df.reset_index()
df.head()

Отчет создан успешно


,Date,CampaignName,AdGroupName,LocationOfPresenceName,AdNetworkType,Device,Impressions,Clicks,Cost,Bounces,AvgPageviews,Conversions
0,2022-01-01,Test API Sandbox campaign 1,Группа №4442697,Москва и область,AD_NETWORK,DESKTOP,2103,207,158.10,0,3.05,--
1,2022-01-01,Test API Sandbox campaign 1,Группа №4442697,Москва и область,AD_NETWORK,MOBILE,734,11,5.25,0,2.75,--
2,2022-01-01,Test API Sandbox campaign 1,Группа №4442697,Москва и область,AD_NETWORK,TABLET,306,54,21.60,0,3.02,--
3,2022-01-01,Test API Sandbox campaign 1,Группа №4442697,Москва и область,SEARCH,DESKTOP,3950,364,312.75,0,2.89,--
4,2022-01-01,Test API Sandbox campaign 1,Группа №4442697,Москва и область,SEARCH,MOBILE,4242,362,243.60,0,2.33,--


In [50]:
# ВАРИАНТ ДЛЯ ВЫГРУЗКИ В БАЙТАХ, НЕОБХОДИМО МЕНЯТЬ СКРИПТ И ВОЗВРАТ ФУНКЦИИ
# присваеваем результат в переменную
#result = req_to_direct(ReportsURL_SANDBOX, body, headers)
# записываем его в байтах
#file_from_bytes = BytesIO(result.content)
# преобразуем в датафрейм
#df = pd.read_csv(file_from_bytes, sep='\t', header=1)
# корректируем значение столбца стоимости при необходимости
# df['Cost'] = df['Cost'] / 1000000 
#df.head()

Отчет создан успешно


AttributeError: 'str' object has no attribute 'content'

## *Функция группировки данных по одному параметру (group_df)*

In [ ]:
def group_df(df, column):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby(column).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']
    
    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
    
    return(grouped_device)

## *Функция группировки по двум пар-м (group_df_2_param)*

In [ ]:
def group_df_2_param(df, column1, column2):
    # сгруппируем количество показов, кликов, СТР и стоимости по устройствам
    grouped_device = df.groupby([column1, column2] ).agg({'visits': ['sum'], 'bounceRate': ['mean'], 
                                             'pageDepth': ['mean'], 'avgVisitSec': ['mean'], 'Goals_total': ['sum']}).reset_index()
    grouped_device.columns = [column1, column2, 'Визитов', '% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Достигнуто целей']

    grouped_device['Конверсия'] = (grouped_device['Достигнуто целей'] / grouped_device['Визитов']) * 100
    for col in ('% откзов', 'Ср глубина просмотра', 'Ср сессия в сек', 'Конверсия'):
        grouped_device[col] = round((grouped_device[col]), 2)
        
    return(grouped_device)